In [24]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [25]:
# Load data from CSV files
kotak_data = pd.read_csv("D:\\stock\\kotak.csv")
fyers_data = pd.read_csv("D:\\stock\\fyers.csv")
angel_data = pd.read_csv("D:\\stock\\angelone.csv")
zerodha_data = pd.read_csv("D:\\stock\\zerodha.csv")

C:\Users\Dell\AppData\Local\Temp\ipykernel_9116\2771533027.py:2: DtypeWarning: Columns (2,11) have mixed types. Specify dtype option on import or set low_memory=False.
  kotak_data = pd.read_csv("D:\\deepak\\akash\\stock\\kotak.csv")
C:\Users\Dell\AppData\Local\Temp\ipykernel_9116\2771533027.py:3: DtypeWarning: Columns (5,8) have mixed types. Specify dtype option on import or set low_memory=False.
  fyers_data = pd.read_csv("D:\\deepak\\akash\\stock\\fyers.csv")
C:\Users\Dell\AppData\Local\Temp\ipykernel_9116\2771533027.py:4: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  angel_data = pd.read_csv("D:\\deepak\\akash\\stock\\angelone.csv")


In [26]:
# Rename columns for clarity
kotak_data.rename(columns={'instrumentName': 'Kotak Symbol', 'name': 'Instrument Name', 'isin': 'ISIN'}, inplace=True)
fyers_data.rename(columns={'Underlying symbol': 'Fyers Symbol', 'Symbol Details': 'Instrument Name'}, inplace=True)
angel_data.rename(columns={'symbol': 'Angel One Symbol', 'name': 'Instrument Name'}, inplace=True)
zerodha_data.rename(columns={'tradingsymbol': 'Zerodha Symbol', 'name': 'Instrument Name'}, inplace=True)


In [27]:
# Drop the 'instrumentToken' column from kotak_data
kotak_data.drop(columns=['instrumentToken'], inplace=True)

In [28]:
# Drop specified columns from fyers_data
fyers_data.drop(columns=['Fytoken', 'Underlying FyToken', 'Reserved column', 'Reserved column.1', 'Reserved column.2'], inplace=True)


In [29]:
# Normalize Instrument Name for consistency
for df in [kotak_data, fyers_data, angel_data, zerodha_data]:
    df['Instrument Name'] = df['Instrument Name'].str.strip().str.upper()


In [30]:
# Filter out rows with NaN ISIN in both kotak_data and fyers_data
kotak_data = kotak_data[kotak_data['ISIN'].notna()]
fyers_data = fyers_data[fyers_data['ISIN'].notna()]


In [31]:
merged_df = pd.merge(kotak_data, fyers_data, on='ISIN', how='inner', suffixes=('_Kotak', '_Fyers'))


In [32]:
merged_df.columns = [f"{col}_kotak" if col in kotak_data.columns and col != 'ISIN' else f"{col}_fyers" if col in fyers_data.columns and col != 'ISIN' else col for col in merged_df.columns]


In [33]:
merged_df.columns

Index(['Kotak Symbol_kotak', 'Instrument Name_Kotak', 'lastPrice_kotak',
       'expiry_kotak', 'strike_kotak', 'tickSize_kotak', 'lotSize_kotak',
       'instrumentType_kotak', 'segment_kotak', 'exchange_kotak', 'ISIN',
       'multiplier_kotak', 'exchangeToken_kotak', 'optionType_kotak',
       'Instrument Name_Fyers', 'Exchange Instrument type_fyers',
       'Minimum lot size_fyers', 'Tick size_fyers', 'Trading Session_fyers',
       'Last update date_fyers', 'Expiry date_fyers', 'Symbol ticker_fyers',
       'Exchange_fyers', 'Segment_fyers', 'Scrip code_fyers',
       'Fyers Symbol_fyers', 'Underlying scrip code_fyers',
       'Strike price_fyers', 'Option type_fyers'],
      dtype='object')

In [34]:
merged_df.sample(5)

,Kotak Symbol_kotak,Instrument Name_Kotak,lastPrice_kotak,expiry_kotak,strike_kotak,tickSize_kotak,lotSize_kotak,instrumentType_kotak,segment_kotak,exchange_kotak,...,Last update date_fyers,Expiry date_fyers,Symbol ticker_fyers,Exchange_fyers,Segment_fyers,Scrip code_fyers,Fyers Symbol_fyers,Underlying scrip code_fyers,Strike price_fyers,Option type_fyers
8063,KCLINFRA,KCL INFRA PROJECTS LTD,3.00,0,0.0,0.01,1,EQ,CASH,BSE,...,2024-10-25,NaN,BSE:KCLINFRA-X,12,10,531784,KCLINFRA,531784,-1.0,XX
5175,STEELCAS,STEELCAST LTD,348.40,0,0.0,0.05,1,EQ,CASH,NSE,...,2024-10-25,NaN,NSE:STEELCAS-EQ,10,10,6803,STEELCAS,6803,-1.0,XX
1955,GULFPETRO,GP PETROLEUMS LTD,45.35,0,0.0,0.05,1,EQ,CASH,NSE,...,2024-10-25,NaN,BSE:GULFPETRO-B,12,10,532543,GULFPETRO,532543,-1.0,XX
4667,RUCHIRA,RUCHIRA PAPERS LTD,115.15,0,0.0,0.05,1,EQ,CASH,NSE,...,2024-10-25,NaN,NSE:RUCHIRA-EQ,10,10,13821,RUCHIRA,13821,-1.0,XX
8393,PURSHOTTAM,PURSHOTTAM INVESTOFIN LTD,17.00,0,0.0,0.05,1,EQ,CASH,BSE,...,2024-10-25,NaN,BSE:PURSHOTTAM-X,12,10,538647,PURSHOTTAM,538647,-1.0,XX


In [35]:
def jaccard_similarity(a, b):
    a_tokens = set(a.split())
    b_tokens = set(b.split())
    return len(a_tokens & b_tokens) / len(a_tokens | b_tokens)


In [36]:
# Function for TF-IDF with cosine similarity and Jaccard similarity
def tfidf_cosine_similarity_with_jaccard(main_df, match_df, main_key, match_key, output_key, cos_threshold=0.5, jaccard_threshold=0.3):
    main_names = main_df[main_key].dropna().unique()
    match_names = match_df[match_key].dropna().unique()
    
    if len(main_names) == 0 or len(match_names) == 0:
        return pd.DataFrame(columns=[main_key, output_key])
    
    all_names = pd.concat([pd.Series(main_names), pd.Series(match_names)]).unique()
    
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(all_names)
    
    main_vectors = tfidf_matrix[:len(main_names)]
    match_vectors = tfidf_matrix[len(main_names):]
    
    similarity_matrix = cosine_similarity(match_vectors, main_vectors)
    
    matched_pairs = []
    for idx, row in enumerate(similarity_matrix):
        best_match_idx = row.argmax()
        best_score = row[best_match_idx]
        if best_score >= cos_threshold:
            main_name = main_names[best_match_idx]
            match_name = match_names[idx]
            # Apply Jaccard similarity as a secondary check
            jaccard_score = jaccard_similarity(main_name, match_name)
            if jaccard_score >= jaccard_threshold:
                matched_pairs.append({
                    main_key: main_name,
                    output_key: match_name
                })
    return pd.DataFrame(matched_pairs)


In [37]:
# Drop the 'token' column from angel_data
angel_data.drop(columns=['token'], inplace=True)
angel_data.columns = [f"{col}_angel" if col != 'Instrument Name' else col for col in angel_data.columns]


In [38]:
# Drop 'instrument_token' and 'exchange_token' from zerodha_data and add '_zerodha' suffix
zerodha_data.drop(columns=['instrument_token', 'exchange_token'], inplace=True)
zerodha_data.columns = [f"{col}_zerodha" if col != 'Instrument Name' else col for col in zerodha_data.columns]


In [39]:
# Verify that 'Instrument Name' exists in all DataFrames
print("Columns in angel_data:", angel_data.columns)
print("Columns in zerodha_data:", zerodha_data.columns)
print("Columns in merged_df:", merged_df.columns)

Columns in angel_data: Index(['Angel One Symbol_angel', 'Instrument Name', 'expiry_angel',
       'strike_angel', 'lotsize_angel', 'instrumenttype_angel',
       'exch_seg_angel', 'tick_size_angel'],
      dtype='object')
Columns in zerodha_data: Index(['Zerodha Symbol_zerodha', 'Instrument Name', 'last_price_zerodha',
       'expiry_zerodha', 'strike_zerodha', 'tick_size_zerodha',
       'lot_size_zerodha', 'instrument_type_zerodha', 'segment_zerodha',
       'exchange_zerodha'],
      dtype='object')
Columns in merged_df: Index(['Kotak Symbol_kotak', 'Instrument Name_Kotak', 'lastPrice_kotak',
       'expiry_kotak', 'strike_kotak', 'tickSize_kotak', 'lotSize_kotak',
       'instrumentType_kotak', 'segment_kotak', 'exchange_kotak', 'ISIN',
       'multiplier_kotak', 'exchangeToken_kotak', 'optionType_kotak',
       'Instrument Name_Fyers', 'Exchange Instrument type_fyers',
       'Minimum lot size_fyers', 'Tick size_fyers', 'Trading Session_fyers',
       'Last update date_fyers', 'Ex

In [40]:
# Apply similarity function using 'Instrument Name_Fyers' from merged_df
matched_angel_df = tfidf_cosine_similarity_with_jaccard(
    main_df=merged_df,
    match_df=angel_data,
    main_key='Instrument Name_Fyers',
    match_key='Instrument Name',
    output_key='Instrument Name_angel'
)

In [41]:

# Apply similarity function using 'Instrument Name_Fyers' from merged_df for zerodha_data
matched_zerodha_df = tfidf_cosine_similarity_with_jaccard(
    main_df=merged_df,
    match_df=zerodha_data,
    main_key='Instrument Name_Fyers',
    match_key='Instrument Name',
    output_key='Instrument Name_zerodha'
)

In [42]:
final_df = pd.merge(merged_df, matched_angel_df, left_on='Instrument Name_Fyers', right_on='Instrument Name_Fyers', how='left')


In [43]:
final_df = pd.merge(final_df, matched_zerodha_df, left_on='Instrument Name_Fyers', right_on='Instrument Name_Fyers', how='left')


In [44]:
final_df = pd.merge(final_df, angel_data, left_on='Instrument Name_Fyers', right_on='Instrument Name', how='left')


In [45]:
final_df = pd.merge(final_df, zerodha_data, left_on='Instrument Name_Fyers', right_on='Instrument Name', how='left')


In [46]:
final_df.to_csv("D:\\deepak\\akash\\stock\\temp.csv")

In [47]:
final_df.columns

Index(['Kotak Symbol_kotak', 'Instrument Name_Kotak', 'lastPrice_kotak',
       'expiry_kotak', 'strike_kotak', 'tickSize_kotak', 'lotSize_kotak',
       'instrumentType_kotak', 'segment_kotak', 'exchange_kotak', 'ISIN',
       'multiplier_kotak', 'exchangeToken_kotak', 'optionType_kotak',
       'Instrument Name_Fyers', 'Exchange Instrument type_fyers',
       'Minimum lot size_fyers', 'Tick size_fyers', 'Trading Session_fyers',
       'Last update date_fyers', 'Expiry date_fyers', 'Symbol ticker_fyers',
       'Exchange_fyers', 'Segment_fyers', 'Scrip code_fyers',
       'Fyers Symbol_fyers', 'Underlying scrip code_fyers',
       'Strike price_fyers', 'Option type_fyers', 'Instrument Name_angel',
       'Instrument Name_zerodha', 'Angel One Symbol_angel',
       'Instrument Name_x', 'expiry_angel', 'strike_angel', 'lotsize_angel',
       'instrumenttype_angel', 'exch_seg_angel', 'tick_size_angel',
       'Zerodha Symbol_zerodha', 'Instrument Name_y', 'last_price_zerodha',
       'exp

In [48]:
def find_mapping(final_df, search_term, broker='Kotak'):
    """
    Search for a script in final_df and return the mapping for all brokers.
    
    Parameters:
    - final_df: The DataFrame containing the combined script master data.
    - search_term: The identifier (e.g., symbol) to search for.
    - broker: The broker column to search in (default is 'Kotak').

    Returns:
    - A DataFrame row containing the mapping for the selected script.
    """
    if broker == 'Kotak':
        result = final_df[final_df['Kotak Symbol_kotak'] == search_term]
    elif broker == 'Fyers':
        result = final_df[final_df['Fyers Symbol_fyers'] == search_term]
    elif broker == 'Angel':
        result = final_df[final_df['Angel One Symbol_angel'] == search_term]
    elif broker == 'Zerodha':
        result = final_df[final_df['Zerodha Symbol_zerodha'] == search_term]
    else:
        raise ValueError("Unsupported broker. Choose from 'Kotak', 'Fyers', 'Angel', 'Zerodha'.")

    if not result.empty:
        return result
    else:
        return "Script not found in the selected broker's data."


In [50]:
search_term = '20MICRONS'  # Replace with an actual symbol from your data
broker = 'Kotak'  # The broker column to search in

result = find_mapping(final_df, search_term, broker)
print(result)


  Kotak Symbol_kotak Instrument Name_Kotak  lastPrice_kotak expiry_kotak  \
0          20MICRONS        20 MICRONS LTD            73.85            0   
1          20MICRONS        20 MICRONS LTD            73.85            0   
2          20MICRONS        20 MICRONS LTD            74.00            0   
3          20MICRONS        20 MICRONS LTD            74.00            0   

   strike_kotak  tickSize_kotak  lotSize_kotak instrumentType_kotak  \
0           0.0            0.05              1                   EQ   
1           0.0            0.05              1                   EQ   
2           0.0            0.05              1                   EQ   
3           0.0            0.05              1                   EQ   

  segment_kotak exchange_kotak  ... Zerodha Symbol_zerodha  Instrument Name_y  \
0          CASH            NSE  ...                    NaN                NaN   
1          CASH            NSE  ...                    NaN                NaN   
2          CASH     